<a href="https://colab.research.google.com/github/ShangWeiKuo/106-2-R-project/blob/master/Code/ResearchMethod/health/5day/LSTM_health_mse_rmse_w5day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from math import sqrt
import numpy as np
from numpy import concatenate, hstack
from matplotlib import pyplot
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat, Timestamp
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU, LeakyReLU
from keras.optimizers import Adam, SGD
from keras.layers import Bidirectional
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
import random as python_random
import os
import matplotlib.dates as mdates
import datetime

In [ ]:
# 引進中文字體檔案
import matplotlib.font_manager as fm
path = os.path.join(os.getcwd(), '/content/drive/MyDrive/paper/SimHei.ttf')
fontprop = fm.FontProperties(fname=path, size=12)

In [ ]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

In [ ]:
from keras import backend as K

session_conf = tf.compat.v1.ConfigProto(
      intra_op_parallelism_threads=1,
      inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(config=session_conf)
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=session_conf))

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# 數據預處理
path = os.path.join(os.getcwd(), '/content/drive/MyDrive/paper/raw_covid_data_v2.csv')
dataset = read_csv(path, index_col=0)
dataset.index.name = "Datetime"
dataset.index = pd.to_datetime(dataset.index)
dataset = dataset[['new_case', 'positive', 'death', 'new_test']]
values = dataset.values

In [ ]:
dataset.values.shape

(414, 4)

In [ ]:
n_days = 5
n_features = 4

reframed = series_to_supervised(values, n_days, 1)

In [ ]:
# 標準化
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# 轉成監督式數據
reframed = series_to_supervised(scaled, n_days, 1)
print(reframed.shape)

# 數據準備
values = reframed.values

# 劃分訓練數據與測試數據 (Train:80%, Test:20%)
n_obs = n_days * n_features
train = values[:331, :]
test = values[331:, :]

# Split into inputs and outputs
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_days, n_features))
test_X = test_X.reshape((test_X.shape[0], n_days, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

# 建模 - 8 layers
model_8 = Sequential()
model_8.add(LSTM(200, activation='tanh', return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
# model_8.add(LeakyReLU())
model_8.add(LSTM(200, activation='tanh', return_sequences=True))
model_8.add(LSTM(200, activation='tanh', return_sequences=True))
model_8.add(LSTM(200, activation='tanh', return_sequences=True))
model_8.add(LSTM(200, activation='tanh'))
model_8.add(Dense(1))
model_8.compile(optimizer='adam', loss='mse')
model_8.summary()
# fit network
history = model_8.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=10, batch_size=32, verbose=2,
                    shuffle=False, callbacks=es)

(409, 24)
(331, 20) 331 (331,)
(331, 5, 4) (331,) (78, 5, 4) (78,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 200)            164000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 200)            320800    
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 200)            320800    
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 200)            320800    
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 1,447,401
Trainable params: 1,447,401
Non

In [ ]:
# make the prediction_8
testPredict_8 = model_8.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_days * n_features))

# inverse the prediction_8
inv_testPredict_8 = concatenate((testPredict_8, test_X[:, -(n_features - 1):]), axis=1)
inv_testPredict_8 = scaler.inverse_transform(inv_testPredict_8)
inv_testPredict_8 = inv_testPredict_8[:, 0]

test_y = test_y.reshape((len(test_y), 1))
inv_ytest = concatenate((test_y, test_X[:, -(n_features - 1):]), axis=1)
inv_ytest = scaler.inverse_transform(inv_ytest)
inv_ytest = inv_ytest[:, 0]

PredictionData = inv_testPredict_8
TrueData = inv_ytest

In [ ]:
pd.DataFrame(PredictionData).to_csv("/content/drive/MyDrive/paper/health/MSE_PredictionData_Test_health_w5day.csv")
pd.DataFrame(TrueData).to_csv("/content/drive/MyDrive/paper/health/MSE_TrueData_health_w5day.csv")